SCRAP "PAGES JAUNES" Results (Professionnal French directory)

In [ ]:
#import usual packages
import numpy as np
import pandas as pd
import requests as rq
import os
import re

In [ ]:
#call the page and display the response => bullshit
page = rq.get('https://www.pagesjaunes.fr/annuaire/departement/nord-59/assurances-agents-generaux')
page_content = page.content
page_content

BEAUTIFOUL SOUP (tests)

In [ ]:
#use BeautifulSoup to try better => bullshit
from bs4 import BeautifulSoup
page_BS = BeautifulSoup(page_content, 'html.parser')
page_BS

In [ ]:
# explore html elements and find name

# list of a tags
list_a = page_BS.find_all('a')

# extract a tags text value
aText_list = [tag for tag in page_BS.find_all('a') if any(re.match(r'bi-denomination pj-link', attr) for attr in tag.get('class', []))]

for aText in aText_list:
    print(aText)

# list h3 tags
list_h3 = page_BS.select('div.bi-with-visual div.bi-content div.bi-header-title a.bi-denomination pj-link h3')
list_h3

SELENIUM

In [ ]:
#import and parameter

from selenium import webdriver

options = webdriver.ChromeOptions()
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
service = Service(executable_path = 'C:/Users/BGE/Google Drive/Visual Studio Code/chromedriver-win64/chromedriver.exe') 

driver = webdriver.Chrome(service=service, options=options)

driver.get('https://www.pagesjaunes.fr/annuaire/departement/nord-59/assurances-agents-generaux')

#submodules and classes within selenium.webdriver.common
from selenium.webdriver.common.by import By

#set up Webdriverwait and support classes
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#access to dataPrivacy_button and click to agree on main page 

OkButtonPrivacy = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#didomi-notice-agree-button')))
OkButtonPrivacy.click()






In [ ]:
# Input city name for search

from selenium.webdriver.common.keys import Keys

# Wait for the elements to be present
wait = WebDriverWait(driver, 20)

input_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#ou')))

# input search words and press 'return' button
input_element.send_keys('croix 59170' + Keys.RETURN)  # Pressing Enter after inputting the city name


# 1st attempt to display names
names_a_list = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.bi-clic-mobile > div > div > div.bi-header-title > a')))

# Finding all 'li' tags with an id that starts with 'epj-' // html elt parent of names tags
li_tags = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//*[starts-with(@id, 'epj')]"))    )
    

# Navigate in reluts list => click next page button

# find how many pages in total => select webElt within is mentionned the total nb of pages

next_button_counter = driver.find_elements(By.CSS_SELECTOR, '.pagination > #SEL-compteur')

# extract chararacters in pages counter to obtain total nb of pages ( 'xx / 20' for e.g.)
paging_text = next_button_counter[0].text


# extract nb of pages by extracting last characters from "/" until end => Regex

import re

# Define a pattern to capture the last two characters followed by a backslash
pattern = r'\d+$'
total_pages_text = re.findall(pattern, paging_text)
# select counter on first page of results
total_pages_text = int(total_pages_text[0])


In [ ]:
# create a table with list of results (name / postal address / telephone nb)

name_insurers_agg = []
address_insurers_agg = []
telNb_insurers_agg = []


# iterate on every page results
for total_page_text in range(1, total_pages_text + 1):

    h3_tags = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.bi-clic-mobile > div > div > div.bi-header-title > a > h3"))) 
# Extract the text from each h3 element = names    
    elements_text = [h3.text.strip() for h3 in h3_tags]
    for text in elements_text:
        name_insurers_agg.append(text)

# Create a list of "a" webElt 
    aText_tags = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.bi-clic-mobile > div > div > div.bi-address.small > a")))
# Extract text from a elements = addresses        
    address_text = [aText.text.strip() for aText in aText_tags]
    for address in address_text:
        address_insurers_agg.append(address)

# import Action chains class    
    from selenium.webdriver.common.action_chains import ActionChains
# Move to the button before clicking
    # import package to manage interactions 
    actions = ActionChains(driver)

    #import "JavascriptException" packages to use try / except statement
    from selenium.common.exceptions import JavascriptException

    # create a list of all button to push for displaying telephone numbers
    telNb_buttons = WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.bi-ctas > button')))
    

    # try / except to displays possibly errors
    try: 
        # Wait for the element to be visible
        for telNb_button in telNb_buttons:
            # interact with the element 
            actions.move_to_element(telNb_button).perform()
            telNb_button.click()
          
    except JavascriptException as e:
        print(f"JavaScript exception encountered: {e}")


# extract phone numbers: hardest task because of seceral display cases of this nb in different webElt and positions in the DOM

    tel_numbers_list = []
   
    # Wait and find the parent elements
    parent_elements = WebDriverWait(driver, 5).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.number-contact:nth-child(1)')))

    # Iterate through each parent element
    for parent_element in parent_elements:
        # Find all child span elements within the parent element
        child_spans = parent_element.find_elements(By.CSS_SELECTOR, 'span')
        # if span elt does exist:
        if child_spans:         
            for child_span in child_spans: 
                # check if it's really a webElt to iterate with tailored methods
                if isinstance(child_span, webdriver.remote.webelement.WebElement):
                # if webElt, but no txt within span 
                    if not child_span.text :
                        tel_numbers_list.append('None1')
                # if text within span, check if fits with good relevant class or none class (cases which fit with our searched info)
                    elif 'annonceur' in child_span.get_attribute('class') or not child_span.get_attribute('class') :
                        tel_number = child_span.text.strip()
                    # append all number in a list
                        tel_numbers_list.append(tel_number)
        # if span elt does not exist:
        else: 
            tel_numbers_list.append('None2')

    # (a bit duplicate? Y) terate tel numbers list and append to final list, with a text if no number
    for tel_number in tel_numbers_list:
        if tel_number:
            telNb_insurers_agg.append(tel_number)
        else:
            tel_numbers_list.append('Holy2 SHit')
        

    # Do this actions on evry page reults => Locate and click the "next" button if not on the last page
    if total_page_text < total_pages_text:
        next_button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a#pagination-next')))
        next_button.click()
    else:
        print("Reached the last page")



# Optional: Add a delay to ensure the page fully loads before extracting results
import time
time.sleep(2)

# Close the WebDriver
driver.quit()



In [ ]:
# do tests and due diligence before to export

print(len(name_insurers_agg))
name_insurers_agg[17]
address_insurers_agg[17]

# transform in DataFrame => pandas
pd.DataFrame(name_insurers_agg)
pd.DataFrame(address_insurers_agg)
pd.DataFrame(telNb_insurers_agg)

# remove tel nb from rows within telephone number is marked up with 'Opposé aux opérations de marketing'
telNb_insurers_agg = ['None' if x == '' else x for x in telNb_insurers_agg if x != 'Opposé aux opérations de marketing' and x != '']

# check equal numbers of rows in the three lists we want to merge
print(f"nb noms : {len(name_insurers_agg)}")
print(f"nb adresses: {len(address_insurers_agg)}")
print(f"nb tel: {len(telNb_insurers_agg)}")

# merge the three lists to create a unified table
insurers_list = {'insurers': name_insurers_agg,
                 'addresses' : address_insurers_agg,
                 'tel': telNb_insurers_agg}

insurers_list = pd.DataFrame(insurers_list)

# Erase 'Voir le plan' => Replace on all selected columns
insurers_list['addresses'] = insurers_list['addresses'].apply(lambda x: x.replace('Voir le plan', ''))

# Set a numerical index
insurers_list.index.name = 'Index'

# display the final list
insurers_list

In [ ]:
# save the result in csv format
insurers_list.to_csv('leads_insurers_list_RbxAndAround_082024.csv')